In [2]:
import pandas as pd
import os


In [3]:
# ฟังก์ชันประมวลผลไฟล์ CSV เดี่ยว
def process_csv(file_path):
    """
    ประมวลผลไฟล์ CSV เดี่ยว:
    - เติมค่า city และ country จาก longitude และ latitude หากมี null
    - ลบแถวที่ยังมี null หลังจากพยายามเติมข้อมูลแล้ว
    """
    df = pd.read_csv(file_path)
    initial_null_count = df[['city', 'country']].isnull().sum().sum()
    fixes_count = 0

    for idx, row in df.iterrows():
        if pd.isnull(row['city']) or pd.isnull(row['country']):
            matched_row = df[(df['longitude'] == row['longitude']) &
                             (df['latitude'] == row['latitude']) &
                             df['city'].notnull() &
                             df['country'].notnull()]
            if not matched_row.empty:
                df.loc[idx, 'city'] = matched_row.iloc[0]['city']
                df.loc[idx, 'country'] = matched_row.iloc[0]['country']
                fixes_count += 1

    df = df.dropna()
    remaining_null_count = df[['city', 'country']].isnull().sum().sum()
    print(f"{file_path}: แก้ไขได้ {fixes_count} แถว, เหลือ null {remaining_null_count} แถว")
    return df

# ฟังก์ชันประมวลผลไฟล์ CSV ทั้งหมดในโฟลเดอร์
def process_all_csv_in_folder(folder_path):
    """
    ประมวลผลไฟล์ CSV ทุกไฟล์ในโฟลเดอร์:
    - เรียก process_csv() สำหรับแต่ละไฟล์
    - รวมข้อมูลทั้งหมดเป็น DataFrame เดียว
    """
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    if not csv_files:
        print(f"ไม่พบไฟล์ CSV ในโฟลเดอร์: {folder_path}")
        return pd.DataFrame()

    processed_dataframes = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        processed_df = process_csv(file_path)
        processed_dataframes.append(processed_df)

    merged_df = pd.concat(processed_dataframes, ignore_index=True)
    return merged_df

# ฟังก์ชันบันทึกผลลัพธ์
def save_to_csv(df, output_file):
    """
    บันทึก DataFrame ลงไฟล์ CSV
    """
    df.to_csv(output_file, index=False)
    print(f"บันทึกผลลัพธ์ในไฟล์: {output_file}")


In [4]:
# กำหนดพาธของโฟลเดอร์และไฟล์ผลลัพธ์
folder_path = "/Users/pichamonjaritngam/Desktop/dsde/prepare"  # แก้ไขพาธที่นี่
output_file = "/Users/pichamonjaritngam/Desktop/dsde/prepare/2013-2017.csv"   # ระบุพาธไฟล์ผลลัพธ์

# ตรวจสอบและเรียกใช้งาน
if os.path.isdir(folder_path):
    merged_df = process_all_csv_in_folder(folder_path)
    if not merged_df.empty:
        save_to_csv(merged_df, output_file)
    else:
        print("ไม่มีข้อมูลให้บันทึก")
else:
    print(f"ไม่พบโฟลเดอร์: {folder_path}")


/Users/pichamonjaritngam/Desktop/dsde/prepare/prepared_2016_data.csv: แก้ไขได้ 0 แถว, เหลือ null 0 แถว
/Users/pichamonjaritngam/Desktop/dsde/prepare/prepared_2017_data.csv: แก้ไขได้ 0 แถว, เหลือ null 0 แถว
/Users/pichamonjaritngam/Desktop/dsde/prepare/prepared_2015_data.csv: แก้ไขได้ 0 แถว, เหลือ null 0 แถว
/Users/pichamonjaritngam/Desktop/dsde/prepare/prepared_2014_data.csv: แก้ไขได้ 0 แถว, เหลือ null 0 แถว
บันทึกผลลัพธ์ในไฟล์: /Users/pichamonjaritngam/Desktop/dsde/prepare/2013-2017.csv
